In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
import random
import open3d as o3d
from torch.utils.data import DataLoader
from learning3d.models import PPFNet
from learning3d.data_utils import ClassificationData, ModelNet40Data
import random

# Cihaz ayarı
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Test veri kümesini yükleme
test_dataset = ClassificationData(data_class=ModelNet40Data(train=False, num_points=4096))
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=2)

# Modeli oluştur ve ağırlıkları yükle
class PPFNetClassifier(nn.Module):
    def __init__(self, num_classes=40, embedding_dim=128):
        super(PPFNetClassifier, self).__init__()
        self.encoder = PPFNet(features=['ppf', 'dxyz', 'xyz'], emb_dims=embedding_dim, radius=0.3, num_neighbors=64)
        self.fc = nn.Sequential(
            nn.Linear(embedding_dim, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(negative_slope=0.1),
            nn.Dropout(0.4),

            nn.Linear(512, 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(negative_slope=0.1),
            nn.Dropout(0.4),

            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(negative_slope=0.1),
            nn.Dropout(0.3),

            nn.Linear(512, num_classes)
        )
        
    def forward(self, points, normals):
        features = self.encoder(points, normals)
        global_embedding = torch.max(features, dim=1)[0]
        return self.fc(global_embedding)

model = PPFNetClassifier(num_classes=40, embedding_dim=128).to(device)
model.encoder.load_state_dict(torch.load("ppfnet_encoder_epoch25.pth", map_location=device))
model.fc.load_state_dict(torch.load("ppfnet_classifier_epoch25.pth", map_location=device))
model.eval()

# Normal hesaplama fonksiyonu
def compute_normals_single(points_single):
    """Open3D kullanarak yüzey normallerini hesaplar."""
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points_single.astype(np.float64))
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    return np.asarray(pcd.normals)

def random_rotation_single_axis(points, max_angle):
    """Belirli bir eksende rastgele dönüşüm uygular."""
    axis = random.choice([0, 1, 2])  # Rastgele bir eksen seç
    angle = random.uniform(-max_angle, max_angle)  # Rastgele dönüş açısı belirle
    
    rotation_matrix = torch.eye(3)  # 3x3 birim matris
    c, s = torch.cos(torch.tensor(angle)), torch.sin(torch.tensor(angle))
    
    if axis == 0:  # X ekseni dönüşü
        rotation_matrix[1, 1] = c
        rotation_matrix[1, 2] = -s
        rotation_matrix[2, 1] = s
        rotation_matrix[2, 2] = c
    elif axis == 1:  # Y ekseni dönüşü
        rotation_matrix[0, 0] = c
        rotation_matrix[0, 2] = s
        rotation_matrix[2, 0] = -s
        rotation_matrix[2, 2] = c
    else:  # Z ekseni dönüşü
        rotation_matrix[0, 0] = c
        rotation_matrix[0, 1] = -s
        rotation_matrix[1, 0] = s
        rotation_matrix[1, 1] = c

    return torch.matmul(points, rotation_matrix.to(points.device))  # Dönüşümü uygula


def random_rotation(points, max_angle):
    """Rastgele dönüşüm uygular."""
    angles = torch.empty(3).uniform_(-max_angle, max_angle)  # Her eksen için rastgele açı seç
    Rx = torch.tensor([
        [1, 0, 0],
        [0, torch.cos(angles[0]), -torch.sin(angles[0])],
        [0, torch.sin(angles[0]), torch.cos(angles[0])]
    ])
    Ry = torch.tensor([
        [torch.cos(angles[1]), 0, torch.sin(angles[1])],
        [0, 1, 0],
        [-torch.sin(angles[1]), 0, torch.cos(angles[1])]
    ])
    Rz = torch.tensor([
        [torch.cos(angles[2]), -torch.sin(angles[2]), 0],
        [torch.sin(angles[2]), torch.cos(angles[2]), 0],
        [0, 0, 1]
    ])
    rotation_matrix = torch.matmul(Rz, torch.matmul(Ry, Rx))  # Döndürme matrisini oluştur
    return torch.matmul(points, rotation_matrix.to(points.device))  # Dönüşümü uygula


def cosine_similarity(vec1, vec2):
    """Cosine similarity hesaplar."""
    return F.cosine_similarity(vec1, vec2, dim=-1).mean().item()

def euclidean_distance(vec1, vec2):
    """Euclidean distance hesaplar."""
    return torch.norm(vec1 - vec2, p=2, dim=-1).mean().item()

def evaluate_encoder_similarity():
    """Encoder çıktılarının benzerlik skorlarını hesaplar."""
    scores = {
        "Single Small": {"cosine": [], "euclidean": []},
        "All Small": {"cosine": [], "euclidean": []},
        "Single Large": {"cosine": [], "euclidean": []},
        "All Large": {"cosine": [], "euclidean": []},
    }

    with torch.no_grad():
        for points, _ in test_loader:
            points = points.to(device)

            # 4 Farklı Dönüşüm
            rotated_1 = random_rotation_single_axis(points, torch.pi / 18)  # Single Small (~10°)
            rotated_2 = random_rotation(points, torch.pi / 18)  # All Small (~10°)
            rotated_3 = random_rotation_single_axis(points, torch.pi / 3)   # Single Large (~60°)
            rotated_4 = random_rotation(points, torch.pi / 3)   # All Large (~60°)

            # Normalize
            normals = torch.tensor(np.stack([compute_normals_single(p.cpu().numpy()) for p in points.cpu()], axis=0), dtype=torch.float32).to(device)
            normals_1 = torch.tensor(np.stack([compute_normals_single(p.cpu().numpy()) for p in rotated_1.cpu()], axis=0), dtype=torch.float32).to(device)
            normals_2 = torch.tensor(np.stack([compute_normals_single(p.cpu().numpy()) for p in rotated_2.cpu()], axis=0), dtype=torch.float32).to(device)
            normals_3 = torch.tensor(np.stack([compute_normals_single(p.cpu().numpy()) for p in rotated_3.cpu()], axis=0), dtype=torch.float32).to(device)
            normals_4 = torch.tensor(np.stack([compute_normals_single(p.cpu().numpy()) for p in rotated_4.cpu()], axis=0), dtype=torch.float32).to(device)

            # **Encoder Çıktıları**
            original_encoding = model.encoder(points, normals)
            original_encoding, _ = torch.max(original_encoding, dim=1)
            enc_1 = model.encoder(rotated_1, normals_1)
            enc_1, _ = torch.max(enc_1, dim=1)
            enc_2 = model.encoder(rotated_2, normals_2)
            enc_2, _ = torch.max(enc_2, dim=1)
            enc_3 = model.encoder(rotated_3, normals_3)
            enc_3, _ = torch.max(enc_3, dim=1)
            enc_4 = model.encoder(rotated_4, normals_4)
            enc_4, _ = torch.max(enc_4, dim=1)

            scores["Single Small"]["cosine"].append(cosine_similarity(original_encoding, enc_1))
            scores["Single Small"]["euclidean"].append(euclidean_distance(original_encoding, enc_1))

            scores["All Small"]["cosine"].append(cosine_similarity(original_encoding, enc_2))
            scores["All Small"]["euclidean"].append(euclidean_distance(original_encoding, enc_2))

            scores["Single Large"]["cosine"].append(cosine_similarity(original_encoding, enc_3))
            scores["Single Large"]["euclidean"].append(euclidean_distance(original_encoding, enc_3))

            scores["All Large"]["cosine"].append(cosine_similarity(original_encoding, enc_4))
            scores["All Large"]["euclidean"].append(euclidean_distance(original_encoding, enc_4))

    # **Ortalama Skorları Hesapla ve Yazdır**
    print("\n==== Ortalama Benzerlik ve Mesafeler ====")
    for key, values in scores.items():
        avg_cosine = sum(values["cosine"]) / len(values["cosine"])
        avg_euclidean = sum(values["euclidean"]) / len(values["euclidean"])
        print(f"{key} - Cosine Similarity: {avg_cosine:.4f}, Euclidean Distance: {avg_euclidean:.4f}")


evaluate_encoder_similarity()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Using device: cuda

==== Ortalama Benzerlik ve Mesafeler ====
Single Small - Cosine Similarity: 0.9999, Euclidean Distance: 0.0207
All Small - Cosine Similarity: 0.9996, Euclidean Distance: 0.0393
Single Large - Cosine Similarity: 0.9980, Euclidean Distance: 0.0822
All Large - Cosine Similarity: 0.9959, Euclidean Distance: 0.1293
